# Predicting the Price of the S&P500 Index with Linear Regression

In [18]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from IPython.display import display

In [3]:
#Read in the data set
df = pd.read_csv(r'C:\Users\user\Desktop\Data Scientist Course\Predicting the stock market\sphist.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [4]:
#Sort by the date column in ascending order
df = df.sort_values("Date", ascending=True)

In [5]:
#Add features that could be helpful for machine learning
df['5 Days Open'] = df['Open'].rolling(window=5).mean()
df['5 Days High'] = df['High'].rolling(window=5).mean()
df['5 Days Low'] = df['Low'].rolling(window=5).mean()
df['5 Days Volume'] = df['Volume'].rolling(window=5).mean()
df['Year'] = df['Date'].apply(lambda x: x.year)

In [6]:
#Adding Day of week column and set it to categorical
df['DOW'] = df['Date'].apply(lambda x: x.weekday())
dow_df = pd.get_dummies(df['DOW'])
df = pd.concat([df, dow_df], axis=1)
df = df.drop(['DOW'], axis=1)

In [7]:
#Because 'rolling' method above include the current date to compute, the current date include future knowledge, which means that the prediction will look not good in real world , so we shift all the values forward one day.
df['5 Days Open'] = df['5 Days Open'].shift(1)
df['5 Days High'] = df['5 Days High'].shift(1)
df['5 Days Low'] = df['5 Days Low'].shift(1)
df['5 Days Volume'] = df['5 Days Volume'].shift(1)

In [8]:
df = df[df['Date'] >= datetime(year=1951, month=1, day=3)]
df.dropna(axis=0)

#Split dataset into train set and test set.
train_df = df[df['Date'] < datetime(year=2013, month=1, day=1)]
test_df = df[df['Date'] >= datetime(year=2013, month=1, day=1)]

In [9]:
#Feature columns
train_colums = ['5 Days Open', '5 Days Volume', '5 Days High', '5 Days Low', 'Year', 0, 1, 2, 3, 4]

In [10]:
# Perform linear regression.
lr = LinearRegression()
lr.fit(train_df[train_colums], train_df['Close'])
prediction = lr.predict(test_df[train_colums])

In [28]:
test_df_copy = test_df.copy()
test_df_copy['Predicted close price'] = prediction
test_df_copy['Difference'] = abs(test_df_copy['Close'] - test_df_copy['Predicted close price'])
test_df_copy['Accuracy(%)'] = (1-(abs(test_df_copy['Close'] - test_df_copy['Predicted close price']) / test_df_copy['Close']
)) * 100

In [31]:
# Error metrics.
mse = mean_squared_error(test_df['Close'], prediction)
rmse = np.sqrt(mse)
test_df_copy = DataFrame(test_df_copy[['Date','Close','Predicted close price','Difference', 'Accuracy(%)']])
print('mse:',mse)
print('rmse:',rmse)
print('-'*60)
print('The S&P500 Index')
display(test_df_copy.sort_values(by=['Date'], ascending=False).head(20))

mse: 378.9356384494125
rmse: 19.466269248354
------------------------------------------------------------
The S&P500 Index


,Date,Close,Predicted close price,Difference,Accuracy(%)
0,2015-12-07,2077.070068,2083.753457,6.683389,99.678230
1,2015-12-04,2091.689941,2069.170510,22.519431,98.923386
2,2015-12-03,2049.620117,2085.905615,36.285498,98.229648
3,2015-12-02,2079.510010,2090.797168,11.287158,99.457220
4,2015-12-01,2102.629883,2081.013283,21.616600,98.971926
5,2015-11-30,2080.409912,2089.912592,9.502680,99.543230
6,2015-11-27,2090.110107,2087.914387,2.195720,99.894947
7,2015-11-25,2088.870117,2097.554893,8.684776,99.584236
8,2015-11-24,2089.139893,2095.813837,6.673944,99.680541
9,2015-11-23,2086.590088,2097.850319,11.260231,99.460352
